# Theano 基础

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

首先导入 `theano` 及其 `tensor` 子模块（`tensor`，张量）：

In [2]:
import theano

# 一般都把 `tensor` 子模块导入并命名为 T
import theano.tensor as T

`tensor` 模块包含很多我们常用的数学操作，所以为了方便，将其命名为 T。

## 符号计算

`theano` 中，所有的算法都是用符号计算的，所以某种程度上，用 `theano` 写算法更像是写数学（之前在[04.06 积分](../04. scipy/04.06 integration in python.ipynb)一节中接触过用 `sympy` 定义的符号变量）。

用 `T.scalar` 来定义一个符号标量：

In [3]:
foo = T.scalar('x')

In [4]:
print foo

x


支持符号计算：

In [5]:
bar = foo ** 2

print bar

Elemwise{pow,no_inplace}.0


这里定义 `foo` 是 $x$，`bar` 就是变量 $x^2$，但显示出来的却是看不懂的东西。

为了更好的显示 `bar`，我们使用 `theano.pp()` 函数（`pretty print`）来显示：

In [6]:
print theano.pp(bar)

(x ** TensorConstant{2})


查看类型：

In [7]:
print type(foo)
print foo.type

<class 'theano.tensor.var.TensorVariable'>
TensorType(float64, scalar)


## theano 函数

有了符号变量，自然可以用符号变量来定义函数，`theano.function()` 函数用来生成符号函数：

    theano.function(input, output)

其中 `input` 对应的是作为参数的符号变量组成的列表，`output` 对应的是输出，输出可以是一个，也可以是多个符号变量组成的列表。

例如，我们用刚才生成的 `foo` 和 `bar` 来定义函数：

In [8]:
square = theano.function([foo], bar)

使用 `square` 函数：

In [9]:
square(3)

array(9.0)

也可以使用 `bar` 的 `eval` 方法，将 `x` 替换为想要的值，`eval` 接受一个字典作为参数，键值对表示符号变量及其对应的值：

In [10]:
bar.eval({foo: 3})

array(9.0)

## theano.tensor

除了 `T.scalar()` 标量之外，`Theano` 中还有很多符号变量类型，这些都包含在 `tensor`（张量）子模块中，而且 `tensor` 中也有很多函数对它们进行操作。

- `T.matrix()` 矩阵
- `T.scalar()` 标量
- `T.vector()` 向量

In [11]:
A = T.matrix('A')
x = T.vector('x')
b = T.vector('b')

`T.dot()` 表示矩阵乘法：
$$y = Ax+b$$

In [12]:
y = T.dot(A, x) + b

`T.sum()` 表示进行求和：
$$z = \sum_{i,j} A_{ij}^2$$

In [13]:
z = T.sum(A**2)

来定义一个线性函数，以 $A,x,b$ 为参数，以 $y,z$ 为输出： 

In [14]:
linear_mix = theano.function([A, x, b],
                             [y, z])

使用这个函数：

$$
A = \begin{bmatrix}
1 & 2 & 3 \\
4 & 5 & 6
\end{bmatrix}, 
x = \begin{bmatrix}
1 \\ 2 \\ 3
\end{bmatrix},
b = \begin{bmatrix}
4 \\ 5
\end{bmatrix}
$$

In [15]:
print linear_mix(np.array([[1, 2, 3],
                           [4, 5, 6]]),    #A
                 np.array([1, 2, 3]),      #x
                 np.array([4, 5]))         #b

[array([ 18.,  37.]), array(91.0)]


我们还可以像定义普通函数一样，给 `theano` 函数提供默认值，需要使用 `theano.Param` 类：

In [16]:
linear_mix_default = theano.function([A, x, theano.Param(b, default=np.zeros(2))],
                                     [y, z])

计算默认参数下的结果：

In [17]:
print linear_mix_default(np.array([[1, 2, 3],
                           [4, 5, 6]]),    #A
                 np.array([1, 2, 3]))      #x

[array([ 14.,  32.]), array(91.0)]


计算刚才的结果：

In [18]:
print linear_mix_default(np.array([[1, 2, 3],
                                   [4, 5, 6]]),    #A
                         np.array([1, 2, 3]),      #x
                         np.array([4, 5]))         #b

[array([ 18.,  37.]), array(91.0)]


## 共享的变量

Theano